## 支持向量机（SVM）算法

这份代码实现了一个基于 **序列最小优化（SMO）算法** 的支持向量机（SVM）分类器。SMO 是一种用于训练支持向量机的有效算法，它将复杂的二次规划问题分解为一系列简单的两变量优化问题，这些子问题可以通过解析方式求解。

### 核心思想

SVM 的目标是找到一个最优的超平面，将不同类别的样本最大程度地分开。这个超平面由法向量 $w$ 和偏置 $b$ 决定。在对偶形式下，这个问题转化为求解拉格朗日乘子 $\alpha_i$，其中只有支持向量对应的 $\alpha_i$ 值不为零。

SMO 算法的核心思想是：
1.  **选择两个 $\alpha_i$ 进行优化**：每次迭代只选择两个拉格朗日乘子 $\alpha_i$ 和 $\alpha_j$ 进行优化，而固定其他所有 $\alpha$。
2.  **解析求解**：由于只涉及两个变量，这个子问题可以被解析地求解，无需使用复杂的二次规划求解器。
3.  **迭代更新**：重复这个过程，直到所有 $\alpha$ 值都满足 KKT（Karush-Kuhn-Tucker）条件，或者达到最大迭代次数。

### 初始化 SVM 模型

`__init__` 方法是 SVM 类的构造函数，用于在创建 SVM 实例时设置模型的**超参数**。

**参数**:
* `C` (float): **惩罚参数**，用于控制模型对误分类的容忍度。C 值越大，对误分类的惩罚越大，模型倾向于选择更窄的间隔以减少训练错误（硬间隔或小容错）。C 值越小，对误分类的惩罚越小，模型允许更多的误分类，间隔可能更宽（软间隔或大容错）。
* `max_iter` (int): **最大迭代次数**，SMO 算法训练过程的最大循环次数。
* `kernel` (str): **核函数类型**，定义数据如何映射到更高维度空间。可选值包括 `'linear'` (线性核), `'poly'` (多项式核), `'rbf'` (径向基函数核)。
* `visualize` (bool): (在此代码中未使用) 是否启用可视化。

In [ ]:
def __init__(self, C=1.0, max_iter=1000, kernel='linear', visualize=False):
        self.max_iter = max_iter  # 最大迭代次数
        self._kernel = kernel  # 核函数类型
        self.C = C  # 惩罚参数C
        self.visualize = visualize  # 是否可视化（此代码中未使用）

### `init_args` 方法：模型参数初始化

`init_args` 方法用于在训练开始前，根据输入的特征和标签数据来**初始化 SVM 模型的内部参数**。

**初始化参数**:
* `self.m`, `self.n`: 样本数量和特征数量。
* `self.X`, `self.Y`: 存储训练特征数据和标签数据。标签 `Y` 被转换为浮点型以方便计算。
* `self.alpha`: **拉格朗日乘子** (Lagrange multipliers)。这是一个大小为 `m` 的向量，初始化为全零。在 SMO 算法中，我们通过优化这些 `alpha` 值来间接确定决策超平面。
* `self.b`: **偏置项** (bias term)，初始化为 0。
* `self.product_matrix`: **核函数矩阵**。这是一个 $m \times m$ 的矩阵，其中 `product_matrix[i][j]` 存储了样本 $x_i$ 和 $x_j$ 之间核函数的值 $K(x_i, x_j)$。在训练前预先计算，可以提高后续计算效率。
* `self.E`: **误差项**。`E_i` 表示模型对样本 $x_i$ 的预测输出 $g(x_i)$ 与真实标签 $y_i$ 之间的差值，即 $E_i = g(x_i) - y_i$。在初始化时，由于所有 `alpha` 都为 0，`g(x_i)` 仅等于 `b`，所以初始 $E_i = b - y_i$。

In [ ]:
def init_args(self, features, labels):
    self.m, self.n = features.shape  # 获取特征矩阵的行数（样本数）和列数（特征数）
    self.X = features  # 存储特征数据
    self.Y = labels.astype(float)  # 存储标签数据，并确保为浮点型以便计算
    self.alpha = np.zeros(self.m)  # 初始化拉格朗日乘子 alpha 为全零向量
    self.b = 0.0  # 初始化偏置项 b 为 0
    self.computer_product_matrix()  # 计算核函数矩阵
    self.E = self.b - self.Y  # 初始化 E 值，E_i = g(x_i) - y_i，初始时 g(x_i) = b

### `computer_product_matrix` 方法：计算核函数矩阵

`computer_product_matrix` 方法负责**预先计算并存储所有训练样本对之间的核函数值**，形成一个对称的核函数矩阵。这可以避免在 SMO 迭代过程中重复计算相同的核函数值，从而显著提高效率。

**计算方式**:
对于任意两个样本 $x_i$ 和 $x_j$，矩阵中的元素 $K_{ij}$ 定义为：
$$K_{ij} = K(x_i, x_j)$$
其中 $K(\cdot, \cdot)$ 是选定的核函数。

In [ ]:
def computer_product_matrix(self):
    # 计算核函数矩阵 K(x_i, x_j)
    self.product_matrix = np.zeros((self.m, self.m), dtype=float)  # 初始化核函数矩阵
    for i in range(self.m):  # 遍历每个样本 i
        for j in range(i, self.m):  # 遍历每个样本 j (只计算上三角部分，然后复制)
            self.product_matrix[i][j] = self.kernel(self.X[i], self.X[j])  # 计算 K(x_i, x_j)
            self.product_matrix[j][i] = self.product_matrix[i][j]  # 填充对称部分

### `kernel` 方法：定义核函数

`kernel` 方法根据 `_kernel` 参数的选择，**实现不同的核函数**。核函数的作用是将原始特征空间的数据映射到更高维的特征空间，使得在原始空间中线性不可分的数据，在高维空间中可能变得线性可分。

**支持的核函数**:
* **线性核 (linear)**:
    $$K(x_1, x_2) = x_1 \cdot x_2$$
    适用于数据在原始空间中近似线性可分的情况。
* **多项式核 (poly)** (本实现为2次多项式):
    $$K(x_1, x_2) = (x_1 \cdot x_2 + 1)^2$$
    可以处理非线性关系，其中 `d` 为多项式次数（本例中 `d=2`）。
* **径向基函数核 (rbf)** (高斯核):
    $$K(x_1, x_2) = \exp(-\gamma \|x_1 - x_2\|^2)$$
    其中 $\gamma$ (gamma) 是一个超参数（本实现中 $\gamma=1$），控制了核函数的宽度。RBF 核能够处理非常复杂的非线性关系，将数据映射到无限维空间。

In [ ]:
def kernel(self, x1, x2):
    # 定义核函数
    if self._kernel == 'linear':  # 线性核函数
        return np.dot(x1, x2)  # 返回点积
    elif self._kernel == 'poly':  # 多项式核函数 (2次)
        return (np.dot(x1, x2) + 1) ** 2  # 返回 (x1·x2 + 1)^2
    elif self._kernel == 'rbf':  # 径向基函数 (RBF) 核
        return np.exp(-1 * np.sum((x1 - x2) ** 2))  # 返回 exp(-||x1-x2||^2)
    return 0  # 默认返回 0

### `function_g` 方法：计算模型的预测输出

`function_g` 方法用于**计算模型对单个样本 $x_i$ 的预测输出 $g(x_i)$**。这个输出的符号决定了样本的分类类别，其绝对值表示样本到决策超平面的距离。

**计算公式**:
对于样本 $x_i$，其预测输出 $g(x_i)$ (或决策函数值) 的计算公式为：
$$g(x_i) = \sum_{j=1}^{m} \alpha_j y_j K(x_j, x_i) + b$$
其中：
* $m$ 是样本总数。
* $\alpha_j$ 是第 $j$ 个样本的拉格朗日乘子。
* $y_j$ 是第 $j$ 个样本的真实标签。
* $K(x_j, x_i)$ 是样本 $x_j$ 和 $x_i$ 之间的核函数值，这里通过预计算的 `product_matrix` 直接获取，即 `self.product_matrix[i][j]`。
* $b$ 是偏置项。

In [ ]:
def function_g(self, i):
    # g(x_i) = sum_j(alpha_j * y_j * K(x_j, x_i)) + b
    return self.b + np.dot((self.alpha * self.Y), self.product_matrix[i])  # 利用预计算的核函数矩阵高效计算

### `create_E` 方法：更新所有误差项

`create_E` 方法用于**高效地重新计算所有训练样本的误差项 $E_i$**。在 SMO 算法中，当 `alpha` 或 `b` 更新后，相应的误差项也需要更新，以便后续的 KKT 条件检查和优化。

**计算公式**:
误差项 $E_i$ 定义为模型对样本 $x_i$ 的预测输出 $g(x_i)$ 与真实标签 $y_i$ 之间的差值：
$$E_i = g(x_i) - y_i$$
该方法通过利用整个核函数矩阵 `product_matrix` 和 `alpha * Y` 的点积，实现对所有 $E_i$ 的并行高效计算：
$$E = (\alpha \cdot Y) \cdot \text{ProductMatrix} + b - Y$$

In [ ]:
def create_E(self):
    # 重新计算所有样本的误差 E 值
    # E_i = g(x_i) - y_i
    self.E = np.dot((self.alpha * self.Y), self.product_matrix) + self.b - self.Y  # 高效计算所有 E 值

### `fit` 方法：训练 SVM 模型

`fit` 方法是 SVM 模型的**训练入口**，它通过**迭代执行 SMO 算法**来优化拉格朗日乘子 $\alpha$ 和偏置项 $b$。

**训练流程**:
1.  **参数初始化**：调用 `init_args` 初始化所有必要的模型参数。
2.  **迭代优化**：
    * 模型会循环 `max_iter` 次（或直到收敛）。
    * 在每次迭代中，会遍历所有训练样本。对于每个样本 $x_{i1}$，调用 `examine_example(i1)` 方法。
    * `examine_example` 会检查当前样本是否违反 KKT 条件。如果违反，它将选择第二个样本 $x_{i2}$，然后调用 `update_alpha_pair(i1, i2)` 来优化这对 $(\alpha_{i1}, \alpha_{i2})$。
3.  **（可选）性能跟踪**：如果提供了测试集 (`X_test`, `y_test`)，则在每次迭代结束时计算模型在测试集上的准确率，并存储起来以便后续分析模型的收敛过程。
4.  **训练完成**：打印训练完成信息，并返回迭代过程中记录的测试准确率列表。

In [ ]:
def fit(self, features, labels, X_test=None, y_test=None):
    # 模型训练方法
    self.init_args(features, labels)  # 初始化参数
    test_accuracies = []  # 存储测试准确率

    for passes in range(self.max_iter):  # 在达到最大迭代次数前循环
        
        # 第一次遍历：遍历所有样本
        for i in range(self.m):  # 遍历所有样本
            self.examine_example(i)  # 检查并尝试优化样本 i

        if X_test is not None and y_test is not None:  # 如果提供了测试集
            current_accuracy = self.score(X_test, y_test)  # 计算当前测试准确率
            test_accuracies.append(current_accuracy)  # 将准确率添加到列表中

    print(f"训练完成，共进行了 {passes+1} 次迭代 (最大迭代次数 {self.max_iter})。")  # 打印训练完成信息
    return test_accuracies  # 返回测试准确率列表

### `examine_example` 方法：检查并选择第二个优化变量

`examine_example` 方法是 SMO 算法的核心部分之一，它负责**检查单个样本 $x_{i1}$ 是否违反 KKT 条件**。如果违反，它将根据启发式规则**选择另一个样本 $x_{i2}$**，并尝试执行两变量优化。

**KKT 条件检查**:
KKT 条件是 SVM 优化问题的必要条件。对于每个样本 $x_i$，其对应的 $\alpha_i$ 必须满足以下条件：
* 如果 $\alpha_i = 0$，则 $y_i g(x_i) \ge 1$ (样本在正确分类的间隔之外或在间隔边界上)。
* 如果 $0 < \alpha_i < C$，则 $y_i g(x_i) = 1$ (样本是支持向量，位于间隔边界上)。
* 如果 $\alpha_i = C$，则 $y_i g(x_i) \le 1$ (样本是支持向量，位于间隔内或在间隔边界上，可能是误分类)。

该方法会计算 `radius = E1 * Y1` (这里 `E1 = g(x_i) - y_i`，所以 `E1 * Y1 = (g(x_i) - y_i) * y_i = y_i g(x_i) - y_i^2 = y_i g(x_i) - 1`)。如果 `radius` 不满足上述 KKT 条件对应的范围，则认为 KKT 条件被违反。

**$i2$ 选择启发式**:
如果 $x_{i1}$ 违反 KKT 条件，算法会尝试选择第二个样本 $x_{i2}$ 进行优化：
1.  **优先选择非边界支持向量**：尝试从那些 `alpha` 值介于 `(0, C)` 之间的样本（即当前处于间隔边界上的支持向量）中选择 $x_{i2}$。选择使得 $|E_1 - E_2|$ 绝对值最大的 $x_{i2}$，这通常能最大化 `alpha` 的优化步长。
2.  **遍历所有样本**：如果上述优先策略没有找到合适的 $i2$ 或者没有成功进行更新，算法会回退到遍历所有样本（随机打乱顺序）来寻找任何一个能与 $x_{i1}$ 共同进行有效优化的 $x_{i2}$。

如果成功更新了 `alpha` 对，函数返回 `True`，否则返回 `False`。

In [ ]:
def examine_example(self, i1):
    """
    检查样本 i1。如果它违反了 KKT 条件，则寻找合适的 i2 并执行优化。
    如果更新了 alpha 对，返回 True, 否则返回 False。
    """
    E1 = self.function_g(i1) - self.Y[i1]  # 重新计算 E1
    
    # 检查 KKT 条件
    radius = E1 * self.Y[i1]  # 计算 y_i * g(x_i) - 1
    
    if (self.alpha[i1] < 1e-6 and radius < 0) or \
        (self.alpha[i1] > self.C - 1e-6 and radius > 0) or \
        (0 + 1e-6 <= self.alpha[i1] <= self.C - 1e-6 and abs(radius) > 1e-6):

        # KKT 条件被违反，尝试寻找 i2
        non_zero_alpha_indices = np.where((self.alpha > 1e-6) & (self.alpha < self.C - 1e-6))[0]
        
        # 尝试优先从支持向量中选择 i2
        if len(non_zero_alpha_indices) > 0:
            j_candidates = [k for k in non_zero_alpha_indices if k != i1]  # 排除 i1
            if len(j_candidates) > 0:
                j2 = j_candidates[np.argmax(np.abs(self.E[j_candidates] - E1))]  # 选择使 |E1-E2| 最大的 i2
                if self.update_alpha_pair(i1, j2):  # 更新 alpha 对
                    return True

        # (2) 如果没有找到，遍历所有样本，只更新一个j2
        all_indices = list(range(self.m))  # 获取所有样本索引
        np.random.shuffle(all_indices)  # 随机打乱索引，增加鲁棒性
        for j2 in all_indices:  # 遍历所有样本
            if j2 == i1:  # 如果 j2 等于 i1，则跳过
                continue
            if self.update_alpha_pair(i1, j2):  # 找到一个更新 alpha 对，停止遍历
                return True

    return False  # 没有为这个 i1 执行更新

### `update_alpha_pair` 方法：执行两变量优化

`update_alpha_pair` 方法是 SMO 算法的**核心优化步骤**，它解析地更新一对拉格朗日乘子 $\alpha_{i1}$ 和 $\alpha_{i2}$。

**优化步骤**:
1.  **保存旧值**：记录优化前 $\alpha_{i1}$ 和 $\alpha_{i2}$ 的值 ( `alpha1_old`, `alpha2_old`)。
2.  **计算剪辑边界 `L` 和 `H`**：
    这是为了确保 $\alpha_{i1}, \alpha_{i2}$ 的新值在 $[0, C]$ 范围内，并且满足 $\alpha_1 y_1 + \alpha_2 y_2 = \text{常数}$ 的约束。
    * 如果 $y_1 = y_2$: $L = \max(0, \alpha_1^{\text{old}} + \alpha_2^{\text{old}} - C)$, $H = \min(C, \alpha_1^{\text{old}} + \alpha_2^{\text{old}})$
    * 如果 $y_1 \ne y_2$: $L = \max(0, \alpha_2^{\text{old}} - \alpha_1^{\text{old}})$, $H = \min(C, C + \alpha_2^{\text{old}} - \alpha_1^{\text{old}})$
3.  **计算 $\eta$**：
    $\eta$ 是优化目标函数中的一个关键项，它表示核函数值的组合：
    $$\eta = K(x_{i1}, x_{i1}) + K(x_{i2}, x_{i2}) - 2K(x_{i1}, x_{i2})$$
    如果 $\eta \le 0$，则表明优化方向不明确或点过于接近，此时跳过更新。
4.  **计算未经剪辑的 $\alpha_2^{\text{new}}$**:
    $$\alpha_2^{\text{new, unc}} = \alpha_2^{\text{old}} + \frac{y_2 (E_1 - E_2)}{\eta}$$
5.  **剪辑 $\alpha_2^{\text{new}}$**：将 $\alpha_2^{\text{new, unc}}$ 限制在 $[L, H]$ 范围内，得到最终的 $\alpha_2^{\text{new}}$。
6.  **计算 $\alpha_1^{\text{new}}$**:
    根据约束 $\alpha_1 y_1 + \alpha_2 y_2 = \text{常数}$，可以推导出 $\alpha_1^{\text{new}}$：
    $$\alpha_1^{\text{new}} = \alpha_1^{\text{old}} + y_1 y_2 (\alpha_2^{\text{old}} - \alpha_2^{\text{new}})$$
7.  **更新偏置项 $b$**:
    根据新的 $\alpha_1^{\text{new}}$ 和 $\alpha_2^{\text{new}}$ 来更新偏置项 $b$。有三种情况：
    * 如果 $0 < \alpha_1^{\text{new}} < C$，则 $b_{\text{new}} = b_{\text{old}} - E_1 - y_1 K(x_{i1}, x_{i1}) (\alpha_1^{\text{new}} - \alpha_1^{\text{old}}) - y_2 K(x_{i2}, x_{i1}) (\alpha_2^{\text{new}} - \alpha_2^{\text{old}})$
    * 如果 $0 < \alpha_2^{\text{new}} < C$，则 $b_{\text{new}} = b_{\text{old}} - E_2 - y_1 K(x_{i1}, x_{i2}) (\alpha_1^{\text{new}} - \alpha_1^{\text{old}}) - y_2 K(x_{i2}, x_{i2}) (\alpha_2^{\text{new}} - \alpha_2^{\text{old}})$
    * 如果 $\alpha_1^{\text{new}}$ 和 $\alpha_2^{\text{new}}$ 都在边界上 (0 或 C)，则 $b_{\text{new}} = (b1_{\text{new}} + b2_{\text{new}}) / 2$
8.  **更新 $\alpha$ 值和误差 $E$**：将计算出的 $\alpha_1^{\text{new}}$ 和 $\alpha_2^{\text{new}}$ 存入 `self.alpha`，并调用 `create_E()` 重新计算所有样本的误差项。

In [ ]:
def update_alpha_pair(self, i1, i2):
    """
    对 alpha_i1 和 alpha_i2 执行序列最小化优化。
    如果 alpha 被更新，返回 True, 否则返回 False。
    """
    if i1 == i2:  # 如果两个索引相同，则不进行更新
        return False

    alpha1_old = self.alpha[i1]  # 存储旧的 alpha1
    alpha2_old = self.alpha[i2]  # 存储旧的 alpha2
    Y1 = self.Y[i1]  # 获取标签 Y1
    Y2 = self.Y[i2]  # 获取标签 Y2
    E1 = self.E[i1]  # 使用缓存的 E1 值
    E2 = self.E[i2]  # 使用缓存的 E2 值

    # 计算剪辑边界 L 和 H
    if Y1 == Y2:  # 如果标签相同
        L = max(0, alpha1_old + alpha2_old - self.C)
        H = min(self.C, alpha1_old + alpha2_old)
    else:  # 如果标签不同
        L = max(0, alpha2_old - alpha1_old)
        H = min(self.C, self.C + alpha2_old - alpha1_old)

    # 计算 eta = K11 + K22 - 2*K12
    eta = self.product_matrix[i1][i1] + self.product_matrix[i2][i2] - 2 * self.product_matrix[i1][i2]
    
    # 避免除以零或非常小的 eta (意味着点非常接近)
    if eta <= 0:  # 使用一个小的容差
        return False

    # 计算新的 alpha2 (未剪辑)
    alpha2_new_unc = alpha2_old + Y2 * (E1 - E2) / eta
    alpha2_new = self.clip_alpha(alpha2_new_unc, L, H)  # 剪辑 alpha2


    # 计算新的 alpha1
    alpha1_new = alpha1_old + Y1 * Y2 * (alpha2_old - alpha2_new)

    # 更新偏置项 b
    b1_new = self.b - E1 - Y1 * self.product_matrix[i1][i1] * (alpha1_new - alpha1_old) \
                - Y2 * self.product_matrix[i2][i1] * (alpha2_new - alpha2_old)
    b2_new = self.b - E2 - Y1 * self.product_matrix[i1][i2] * (alpha1_new - alpha1_old) \
                - Y2 * self.product_matrix[i2][i2] * (alpha2_new - alpha2_old)

    if 0 < alpha1_new < self.C:  # 如果 alpha1 在 (0, C) 之间
        self.b = b1_new  # 使用 b1_new 更新 b
    elif 0 < alpha2_new < self.C:  # 如果 alpha2 在 (0, C) 之间
        self.b = b2_new  # 使用 b2_new 更新 b
    else:  # 都在边界上
        self.b = (b1_new + b2_new) / 2  # 取平均值

    # 更新 alpha 值
    self.alpha[i1] = alpha1_new
    self.alpha[i2] = alpha2_new
    
    self.create_E()  # 重新计算所有 E 值

    return True  # alpha 被更改

### `clip_alpha` 方法：剪辑 $\alpha$ 值

`clip_alpha` 方法是一个辅助函数，用于**将计算出的 $\alpha$ 值限制在其有效的范围 $[L, H]$ 内**。

**剪辑规则**:
对于一个未经剪辑的 `_alpha` 值：
* 如果 `_alpha > H`，则返回 `H`。
* 如果 `_alpha < L`，则返回 `L`。
* 否则，返回 `_alpha` 本身。

In [ ]:
def clip_alpha(self, _alpha, L, H):
    # 剪辑 alpha 值到 [L, H] 范围内
    if _alpha > H:
        return H
    elif _alpha < L:
        return L
    else:
        return _alpha

### `predict` 方法：预测单个样本的类别

`predict` 方法用于**对单个新的数据样本进行分类预测**。

**预测公式**:
模型对样本 $x$ 的预测输出 $g(x)$ 为：
$$g(x) = \sum_{j=1}^{m} \alpha_j y_j K(x_j, x) + b$$
其中 $m$ 是训练样本的数量，$\alpha_j, y_j, x_j$ 是训练集中的拉格朗日乘子、标签和样本，$K(\cdot, \cdot)$ 是核函数，$b$ 是偏置项。
* 如果 $g(x) > 0$，模型预测类别为 `1`。
* 如果 $g(x) \le 0$，模型预测类别为 `-1`。

In [ ]:
def predict(self, data):
    # 预测单个样本的类别
    r = self.b  # 初始化 r 为偏置项 b
    for i in range(self.m):  # 遍历所有样本
        r += self.alpha[i] * self.Y[i] * self.kernel(data, self.X[i])  # 累加 alpha_j * y_j * K(x_j, x)
    return 1 if r > 0 else -1  # 根据 r 的符号返回预测类别

### `score` 方法：计算模型准确率

`score` 方法用于**评估 SVM 模型在给定数据集上的分类准确率**。

**计算公式**:
准确率定义为正确分类的样本数量占总样本数量的比例：
$$\text{Accuracy} = \frac{\text{正确分类的样本数}}{\text{总样本数}}$$
该方法遍历测试集中的每个样本，使用 `predict` 方法进行预测，然后与真实标签进行比较，统计正确预测的数量，最终计算出准确率。

In [ ]:
def score(self, X_test, y_test):
    # 计算模型在测试集上的准确率
    right_count = 0  # 正确预测的数量
    for i in range(len(X_test)):  # 遍历测试集中的每个样本
        result = self.predict(X_test[i])  # 预测样本的类别
        if result == y_test[i]:  # 如果预测正确
            right_count += 1  # 正确计数加 1
    return right_count / len(X_test)  # 返回准确率

### `decision_function` 方法：计算样本到决策边界的距离

`decision_function` 方法计算**一个或多个样本到 SVM 决策超平面的有符号距离**。这个距离的符号表示分类方向，其绝对值表示样本离决策边界的远近。

**计算公式**:
对于一个或多个样本 $X_{\text{eval}}$，其到决策边界的距离 $g(x)$ 计算方式与 `predict` 方法中的 $g(x)$ 相同：
$$g(x) = \sum_{j=1}^{m} \alpha_j y_j K(x_j, x) + b$$
* 对于单个样本，返回一个包含其距离的 NumPy 数组。
* 对于多个样本，返回一个包含所有样本距离的 NumPy 数组。

In [ ]:
def decision_function(self, X_eval):
    # 计算样本到决策边界的距离
    if X_eval.ndim == 1:  # 如果是单个样本
        r = self.b  # 初始化 r 为偏置项 b
        for i in range(self.m):  # 遍历所有样本
            r += self.alpha[i] * self.Y[i] * self.kernel(X_eval, self.X[i])  # 累加 alpha_j * y_j * K(x_j, x)
        return np.array([r])  # 返回距离的数组
    else:  # 如果是多个样本
        decisions = []  # 存储每个样本的距离
        for data_point in X_eval:  # 遍历每个数据点
            r = self.b  # 初始化 r
            for i in range(self.m):  # 遍历所有样本
                r += self.alpha[i] * self.Y[i] * self.kernel(data_point, self.X[i])  # 累加
            decisions.append(r)  # 将距离添加到列表
        return np.array(decisions)  # 返回距离的数组

### `support_vectors_` 属性：获取支持向量

`support_vectors_` 是一个**属性 (property)**，它返回**模型训练后识别出的所有支持向量**。

**支持向量定义**:
在 SVM 中，**支持向量**是那些在优化过程中其拉格朗日乘子 $\alpha_i$ 值非零的训练样本（即 $\alpha_i > 0$）。这些样本是决定决策超平面位置的关键点。在软间隔 SVM 中，$\alpha_i$ 值也可能等于 $C$。这里使用一个小的容差 (1e-6) 来判断 $\alpha_i$ 是否非零。

In [ ]:
@property
def support_vectors_(self):
    # 获取支持向量
    # 支持向量是那些 alpha 值非零的样本
    return self.X[self.alpha > 1e-6]  # 返回 alpha 大于一个很小容差的样本

### `support_vector_indices_` 属性：获取支持向量的索引

`support_vector_indices_` 是另一个**属性 (property)**，它返回**模型训练后识别出的所有支持向量在原始训练数据集中的索引**。

**索引获取**:
它返回一个 NumPy 数组，其中包含所有 `alpha` 值大于一个小容差 (1e-6) 的训练样本的原始索引。

In [ ]:
@property
def support_vector_indices_(self):
    # 获取支持向量的索引
    return np.where(self.alpha > 1e-6)[0]  # 返回 alpha 大于一个很小容差的索引